# 금융문자 분석 경진대회

In [1]:
import os
os.chdir('./data/')

import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pandas import DataFrame
from collections import Counter

# from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [2]:
train = pd.read_csv("train.csv")

train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [3]:
train["smishing"].sum()

18703

In [4]:
train.loc[train.smishing == 1]

,id,year_month,text,smishing
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1
23,26,2017-01,(광고)XXX추가 XXX품 특판 안내문XXX 지점에서 취급하고 있는 여신 XXX...,1
31,36,2017-01,(광고)XXX신용관리 XXX 알고 싶다나의 신용과 재무상태는 직접 관리해야지 누군가...,1
40,45,2017-01,(광고)안녕하세요수신을 희망하지 않으실 경우에는 거부 라는 답장을 주시면 KISA ...,1
50,58,2017-01,(광고)한국citi bank 나의 대출한도와 금리는? 대출때문에 고민하고 있다거나 ...,1
67,77,2017-01,(광고)다사다난했던 병신()년을 보내고 새로운 정유()년에는 늘 좋은 일과 뜻하시는...,1
71,81,2017-01,(광고)한국 (XXX XXX)2분의 시간만 투자하시면 월200만원 SAVE(절감)되...,1
107,121,2017-01,(광고)한국citi bank나의 대출한도와 금리는?대출때문에 고민하고 있다거나 높은...,1
129,145,2017-01,(광고)신용협동조합 안내XXX세요하루하루 힘든업무와 스트레스에 고생많으십니다여유시간...,1
135,151,2017-01,(광고)신협안녕하십니까 하루하루 힘들고 스트레스 받는 대한민국에서 고생이 많으신데 ...,1


In [10]:
test = pd.read_csv("public_test.csv")

test.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


In [11]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [12]:
19646/train.shape[0]

0.0663839564783997

In [13]:
850/12600

0.06746031746031746

In [14]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [15]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [16]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [17]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((12600, 4), (12600, 1), (1626, 1), (1626, 1))

In [18]:
import konlpy
konlpy.__version__

'0.5.1'

In [19]:
from konlpy.tag import *

In [20]:
tokenizer = Hannanum()

C:\Users\admin\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [21]:
train_doc = [(tokenizer.pos(x), y) for x, y in tqdm(zip(train_xx['text'], train_yyy['smishing']))]
test_doc = [(tokenizer.pos(x), y) for x, y in tqdm(zip(test_xx['text'], test_yyy['smishing']))]

12600it [02:12, 94.87it/s] 
1626it [00:29, 54.29it/s]


In [22]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [36]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [24]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [35]:
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
vec_x_test= v.transform(X_test).toarray()

In [26]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_train)
y_train_pred1_one= [ i[1]  for i in y_train_pred1]

y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

In [29]:
submission=pd.read_csv("submission.csv")

submission.head(2)

submission['smishing'] = y_test_pred1_one

submission.to_csv("14th_baseline_multi.csv",index=False)